<img src="ucsb_logo_seal.png"> 

## ML Feature Utilities

### PSTAT 135 / 235: Big Data Analytics
### University of California, Santa Barbara
### Last Updated: Sep 4, 2019

---  

### Sources
Learning Spark, Chapter 11: Machine Learning with MLlib  
https://spark.apache.org/docs/latest/ml-features.html

### OBJECTIVES
- Discuss functions for extracting features from raw data
- Discuss functions for transforming features
- Discuss functions for selecting features

### CONCEPTS AND FUNCTIONS
SparkSession

The MLlib documentation contains a long list of functions in this area. Below we list some of the more important ones:

- Feature Extractors
    - TF-IDF
    - Word2Vec 
    - CountVectorizer  
    
- Feature Transformers
    - Tokenizer
    - StopWordsRemover
    - n-gram
    - Binarizer
    - OneHotEncoder
    - Normalizer
    - StandardScaler
    - MaxAbsScaler
    - Bucketizer
    - VectorAssembler
    - Imputer 
    
- Feature Selectors
    - ChiSqSelector


### Introducing SparkSession

*SparkSession* is an entry point that was created for tasks such as reading data and creating DataFrames.  

From the developers:  

*We have been getting a lot of questions about the relationship between SparkContext, SQLContext, and HiveContext in Spark 1.x. It was really strange to have “HiveContext” as an entry point when people want to use the DataFrame API. In Spark 2.0, we are introducing SparkSession, a new entry point that subsumes SQLContext and HiveContext. For backward compatibility, we keep the Hive and SQL Contexts.*  

For details:  
https://docs.databricks.com/spark/latest/gentle-introduction/sparksession.html


### Tokenizer  

Break text into individual terms (e.g., words). In some languages this is a non-trivial task.  

**Tokenizer Example**

In [1]:
# load pyspark modules
from pyspark.sql import SparkSession
from pyspark.ml.feature import Tokenizer
spark= SparkSession.builder.getOrCreate()

sentenceDataFrame = spark.createDataFrame([
    (0, "Hi I heard about Spark"),
    (1, "I wish Java could use case classes"),
    (2, "Logistic,regression,models,are,neat")
], ["id", "sentence"])

tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
tokenized = tokenizer.transform(sentenceDataFrame)

tokenized.show()


+---+--------------------+--------------------+
| id|            sentence|               words|
+---+--------------------+--------------------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|
|  1|I wish Java could...|[i, wish, java, c...|
|  2|Logistic,regressi...|[logistic,regress...|
+---+--------------------+--------------------+



### StopWordsRemover

In text processing, stop words are words considered uninformative for analysis.  One of the first preprocessing steps is to remove stop words.  
StopWordsRemover takes a sequence of strings and drops all stop words.  Optionally, the function takes a list of stop words.  Default lists are provided for some languages.

**StopWordsRemover Example**

In [2]:
# load pyspark modules
from pyspark.sql import SparkSession
from pyspark.ml.feature import StopWordsRemover
spark = SparkSession.builder.getOrCreate()

sentenceData = spark.createDataFrame([
    (0, ["I", "saw", "the", "red", "balloon"]),
    (1, ["Mary", "had", "a", "little", "lamb"])
], ["id", "raw"])

remover = StopWordsRemover(inputCol="raw", outputCol="filtered")
remover.transform(sentenceData).show(truncate=False)

+---+----------------------------+--------------------+
|id |raw                         |filtered            |
+---+----------------------------+--------------------+
|0  |[I, saw, the, red, balloon] |[saw, red, balloon] |
|1  |[Mary, had, a, little, lamb]|[Mary, little, lamb]|
+---+----------------------------+--------------------+



### N-gram
An n-gram is a sequence of n adjacent tokens (generally words).
These objects can be useful as features in a model (e.g., indicators of presence of an n-gram in texts).

**n-gram Example**

In [3]:
# load pyspark modules
from pyspark.sql import SparkSession
from pyspark.ml.feature import NGram
spark = SparkSession.builder.getOrCreate()

wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])

ngram = NGram(n=2, inputCol="words", outputCol="ngrams")

ngramDataFrame = ngram.transform(wordDataFrame)
ngramDataFrame.show(truncate=True)


+---+--------------------+--------------------+
| id|               words|              ngrams|
+---+--------------------+--------------------+
|  0|[Hi, I, heard, ab...|[Hi I, I heard, h...|
|  1|[I, wish, Java, c...|[I wish, wish Jav...|
|  2|[Logistic, regres...|[Logistic regress...|
+---+--------------------+--------------------+



### Binarizer

Threshold variable to binary (0/1) features.
Useful for presence/absence, for example.
Feature values greater than a given threshold are binarized to 1.0; values equal to or less than the threshold are binarized to 0.0. Both Vector and Double types are supported for inputCol.

**Binarizer Example**

In [9]:
# load pyspark modules
from pyspark.sql import SparkSession
from pyspark.ml.feature import Binarizer
spark = SparkSession.builder.getOrCreate()

continuousDataFrame = spark.createDataFrame([
    (0, 0.1),
    (1, 0.8),
    (2, 0.2)
], ["id", "feature"])

binarizer = Binarizer(threshold=0.5, inputCol="feature", outputCol="binarized_feature")

binarizedDataFrame = binarizer.transform(continuousDataFrame)

print("Binarizer output with Threshold = %f" % binarizer.getThreshold())
binarizedDataFrame.show()

Binarizer output with Threshold = 0.500000
+---+-------+-----------------+
| id|feature|binarized_feature|
+---+-------+-----------------+
|  0|    0.1|              0.0|
|  1|    0.8|              1.0|
|  2|    0.2|              0.0|
+---+-------+-----------------+



### OneHotEncoder

Maps a column of label indices to a column of binary vectors, with at most a single one-value. 
This is the same as dummy coding.
This encoding allows algorithms which expect continuous features, such as Logistic Regression, to use categorical features.

An intermediate step is to use StringIndexer. 

StringIndexer encodes a string column of labels to a column of label indices. The indices are in [0, numLabels), ordered by label frequencies, so the most frequent label gets index 0.

**OneHotEncoder Example**

In [12]:
# load pyspark modules
from pyspark.sql import SparkSession
from pyspark.ml.feature import OneHotEncoder, StringIndexer
spark= SparkSession.builder.getOrCreate()

df = spark.createDataFrame([
    (0, "a"),
    (1, "b"),
    (2, "c"),
    (3, "a"),
    (4, "a"),
    (5, "c")
], ["id", "category"])

# for each level, count freq. val=0 for most freq, then 1, ...
stringIndexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
model = stringIndexer.fit(df)
indexed = model.transform(df)

encoder = OneHotEncoder(inputCol="categoryIndex", outputCol="categoryVec")
encoded = encoder.transform(indexed)
encoded.show()


+---+--------+-------------+-------------+
| id|category|categoryIndex|  categoryVec|
+---+--------+-------------+-------------+
|  0|       a|          0.0|(2,[0],[1.0])|
|  1|       b|          2.0|    (2,[],[])|
|  2|       c|          1.0|(2,[1],[1.0])|
|  3|       a|          0.0|(2,[0],[1.0])|
|  4|       a|          0.0|(2,[0],[1.0])|
|  5|       c|          1.0|(2,[1],[1.0])|
+---+--------+-------------+-------------+



### Normalizer

Transform each feature vector to have unit norm.  The type of norm (*p-norm*) is passed as a parameter, with p=2 the default (Euclidean distance).  Standardizing the features puts them on the same scale, which can be critical for models where scale matters (e.g., k-nearest neighbor).

**Normalizer Example**

In [13]:
from pyspark.ml.feature import Normalizer
from pyspark.ml.linalg import Vectors

dataFrame = spark.createDataFrame([
    (0, Vectors.dense([1.0, 0.5, -1.0]),),
    (1, Vectors.dense([2.0, 1.0, 1.0]),),
    (2, Vectors.dense([4.0, 10.0, 2.0]),)
], ["id", "features"])

# Normalize each Vector using $L^1$ norm.
normalizer = Normalizer(inputCol="features", outputCol="normFeatures", p=1.0)
l1NormData = normalizer.transform(dataFrame)
print("Normalized using L^1 norm")
l1NormData.show()


Normalized using L^1 norm
+---+--------------+------------------+
| id|      features|      normFeatures|
+---+--------------+------------------+
|  0|[1.0,0.5,-1.0]|    [0.4,0.2,-0.4]|
|  1| [2.0,1.0,1.0]|   [0.5,0.25,0.25]|
|  2|[4.0,10.0,2.0]|[0.25,0.625,0.125]|
+---+--------------+------------------+



### StandardScaler

Transforms each feature vector to have mean zero (centering), standard deviation one (scaling).

Parameters

- withStd: True by default. Scales the data to unit standard deviation.
- withMean: False by default. Centers the data with mean before scaling. 

Important Note
If the data is sparse (mostly zeros), centering will make it dense, destroying the sparsity.   
For cases of sparsity, other scaling techniques are preferable, such as MaxAbsScaler.


**StandardScaler Example**

In [5]:
from pyspark.ml.feature import StandardScaler

dataFrame = spark.read.format("libsvm").load("/home/jovyan/UCSB_BigDataAnalytics/data/mllib/sample_libsvm_data.txt")
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures",
                        withStd=True, withMean=True)

# Compute summary statistics by fitting the StandardScaler
scalerModel = scaler.fit(dataFrame)

# Normalize each feature to have mean zero, unit standard deviation.
scaledData = scalerModel.transform(dataFrame)
scaledData.show(5)

+-----+--------------------+--------------------+
|label|            features|      scaledFeatures|
+-----+--------------------+--------------------+
|  0.0|(692,[127,128,129...|[0.0,0.0,0.0,0.0,...|
|  1.0|(692,[158,159,160...|[0.0,0.0,0.0,0.0,...|
|  1.0|(692,[124,125,126...|[0.0,0.0,0.0,0.0,...|
|  1.0|(692,[152,153,154...|[0.0,0.0,0.0,0.0,...|
|  1.0|(692,[151,152,153...|[0.0,0.0,0.0,0.0,...|
+-----+--------------------+--------------------+
only showing top 5 rows



### MaxAbsScaler

Transforms each feature vector to take values in the range [-1, 1] by dividing through by the maximum absolute value in each feature. It does not center the data, and thus does not destroy sparsity.

**MaxAbsScaler Example**

In [6]:
from pyspark.ml.feature import MaxAbsScaler
from pyspark.ml.linalg import Vectors

dataFrame = spark.createDataFrame([
    (0, Vectors.dense([1.0, 0.1, -8.0]),),
    (1, Vectors.dense([2.0, 1.0, -4.0]),),
    (2, Vectors.dense([4.0, 10.0, 8.0]),)
], ["id", "features"])

scaler = MaxAbsScaler(inputCol="features", outputCol="scaledFeatures")

# Compute summary statistics and generate MaxAbsScalerModel
scalerModel = scaler.fit(dataFrame)

# rescale each feature to range [-1, 1].
scaledData = scalerModel.transform(dataFrame)

scaledData.select("features", "scaledFeatures").show()

+--------------+----------------+
|      features|  scaledFeatures|
+--------------+----------------+
|[1.0,0.1,-8.0]|[0.25,0.01,-1.0]|
|[2.0,1.0,-4.0]|  [0.5,0.1,-0.5]|
|[4.0,10.0,8.0]|   [1.0,1.0,1.0]|
+--------------+----------------+



### Bucketizer

Transform a column of continuous features to a column of feature buckets. 

Parameter

splits: with n+1 splits, there are n buckets. A bucket defined by splits x,y holds values in the range [x,y) except the last bucket, which also includes y. Splits should be strictly increasing. Values at -inf, inf must be explicitly provided to cover all Double values; Otherwise, values outside the splits specified will be treated as errors. 

Two examples of splits are Array(Double.NegativeInfinity, 0.0, 1.0, Double.PositiveInfinity) and Array(0.0, 1.0, 2.0).

**Bucketizer Example**

In [7]:
from pyspark.ml.feature import Bucketizer

splits = [-float("inf"), -0.5, 0.0, 0.5, float("inf")]

data = [(-999.9,), (-0.5,), (-0.3,), (0.0,), (0.2,), (999.9,)]
dataFrame = spark.createDataFrame(data, ["features"])

bucketizer = Bucketizer(splits=splits, inputCol="features", outputCol="bucketedFeatures")

# Transform original data into its bucket index.
bucketedData = bucketizer.transform(dataFrame)

print("Bucketizer output with %d buckets" % (len(bucketizer.getSplits())-1))
bucketedData.show()

Bucketizer output with 4 buckets
+--------+----------------+
|features|bucketedFeatures|
+--------+----------------+
|  -999.9|             0.0|
|    -0.5|             1.0|
|    -0.3|             1.0|
|     0.0|             2.0|
|     0.2|             2.0|
|   999.9|             3.0|
+--------+----------------+



### VectorAssembler

VectorAssembler is a transformer that combines a given list of columns into a single vector column. It is useful for combining raw features and features generated by different feature transformers into a single feature vector, in order to train ML models like logistic regression and decision trees.  

VectorAssembler accepts the following input column types: 

- numeric
- boolean
- vector

In each row, the values of the input columns will be concatenated into a vector in the specified order.

**VectorAssembler Example**

In [25]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

dataset = spark.createDataFrame(
    [(0, 18, 1.0, Vectors.dense([0.0, 10.0, 0.5]), 1.0)],
    ["id", "hour", "mobile", "userFeatures", "clicked"])

assembler = VectorAssembler(
    inputCols=["hour", "mobile", "userFeatures"],
    outputCol="features")

output = assembler.transform(dataset)
print("Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'")
output.select("features", "clicked").show(truncate=False)


Assembled columns 'hour', 'mobile', 'userFeatures' to vector column 'features'
+-----------------------+-------+
|features               |clicked|
+-----------------------+-------+
|[18.0,1.0,0.0,10.0,0.5]|1.0    |
+-----------------------+-------+



### Imputer

Imputes missing values using mean (the default) or median in columns where missing values are located.  The inputs columns should be of DoubleType or FloatType. 

Important Note  
Currently Imputer does not support categorical features and possibly creates incorrect values for columns containing 
categorical features.

**Imputer Example**

In [26]:
from pyspark.ml.feature import Imputer

df = spark.createDataFrame([
    (1.0, float("nan")),
    (2.0, float("nan")),
    (float("nan"), 3.0),
    (4.0, 4.0),
    (5.0, 5.0)
], ["a", "b"])

df.show()

+---+---+
|  a|  b|
+---+---+
|1.0|NaN|
|2.0|NaN|
|NaN|3.0|
|4.0|4.0|
|5.0|5.0|
+---+---+



In [27]:
imputer = Imputer(inputCols=["a", "b"], outputCols=["out_a", "out_b"])
model = imputer.fit(df)

model.transform(df).show()

+---+---+-----+-----+
|  a|  b|out_a|out_b|
+---+---+-----+-----+
|1.0|NaN|  1.0|  4.0|
|2.0|NaN|  2.0|  4.0|
|NaN|3.0|  3.0|  3.0|
|4.0|4.0|  4.0|  4.0|
|5.0|5.0|  5.0|  5.0|
+---+---+-----+-----+



### ChiSqSelector

Oftentimes there is a large set of potential features and we need a way to select a “good” subset. 

ChiSqSelector uses the Chi-Squared test of independence for feature selection. It operates on labeled data with categorical features.  

It supports five selection methods: numTopFeatures, percentile, fpr, fdr, fwe: * 

*numTopFeatures* chooses a fixed number of top features according to a chi-squared test. This is akin to yielding the features with the most predictive power.  

See documentation for details

**ChiSqSelector Example**

In [1]:
# load pyspark modules
from pyspark.sql import SparkSession
from pyspark.ml.feature import ChiSqSelector
from pyspark.ml.linalg import Vectors
spark= SparkSession.builder.getOrCreate()

df = spark.createDataFrame([
    (7, Vectors.dense([0.0, 0.0, 18.0, 1.0]), 1.0,),
    (8, Vectors.dense([0.0, 1.0, 12.0, 0.0]), 0.0,),
    (9, Vectors.dense([1.0, 0.0, 15.0, 0.1]), 0.0,)], ["id", "features", "clicked"])

selector = ChiSqSelector(numTopFeatures=2, featuresCol="features",
                         outputCol="selectedFeatures", labelCol="clicked")

result = selector.fit(df).transform(df)

print("ChiSqSelector output with top %d features selected" % selector.getNumTopFeatures())
result.show()

ChiSqSelector output with top 2 features selected
+---+------------------+-------+----------------+
| id|          features|clicked|selectedFeatures|
+---+------------------+-------+----------------+
|  7|[0.0,0.0,18.0,1.0]|    1.0|      [18.0,1.0]|
|  8|[0.0,1.0,12.0,0.0]|    0.0|      [12.0,0.0]|
|  9|[1.0,0.0,15.0,0.1]|    0.0|      [15.0,0.1]|
+---+------------------+-------+----------------+

